*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [211]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [212]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [213]:
df_books.sort_values(by=['isbn'])

,isbn,title,author
254266,0000913154,The Way Things Work: An Illustrated Encycloped...,C. van Amerongen (translator)
215818,0001010565,Mog's Christmas,Judith Kerr
42563,0001046438,Liar,Stephen Fry
112560,0001046713,Twopence to Cross the Mersey,Helen Forrester
42605,000104687X,"T.S. Eliot Reading \The Wasteland\"" and Other ...",T.S. Eliot
...,...,...,...
72808,B000234N76,Falling Angels,Tracy Chevalier
193540,B000234NC6,It Must've Been Something I Ate: The Return of...,Jeffrey Steingarten
131093,B00029DGGO,"Good Wife Strikes Back, The",Elizabeth Buchan
179433,B0002JV9PY,The Blockade Runners,Jules Verne


In [214]:
df_ratings.sort_values(by=['user'])

,user,isbn,rating
9561,2,0195153448,0.0
9562,7,034542252,0.0
9572,8,0771025661,0.0
9580,8,1881320189,7.0
9579,8,1575663937,6.0
...,...,...,...
9560,278854,0553579606,8.0
9554,278854,0375703063,7.0
9553,278854,0316184152,7.0
9555,278854,042516098X,7.0


In [215]:
df_ratings["user"].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: user, Length: 105283, dtype: int64

In [216]:
df_ratings["isbn"].value_counts()

0971880107     2502
0316666343     1295
0385504209      883
0060928336      732
0312195516      723
               ... 
1568656386        1
1568656408        1
1569551553        1
1570081808        1
05162443314       1
Name: isbn, Length: 340556, dtype: int64

In [217]:
number_of_user_ratings = df_ratings.groupby('user')['rating'].count().reset_index().rename(columns = {'rating': 'no_of_ratings'})

In [218]:
number_of_user_ratings.head()

,user,no_of_ratings
0,2,1
1,7,1
2,8,18
3,9,3
4,10,2


In [219]:
number_of_book_ratings = df_ratings.groupby('isbn')['rating'].count().reset_index().rename(columns = {'rating': 'no_of_book_ratings'})

In [220]:
number_of_book_ratings.head()

,isbn,no_of_book_ratings
0,0330299891,2
1,0375404120,2
2,0586045007,1
3,9022906116,2
4,9032803328,1


In [221]:
# make a copy of dataframe
cleaned_ratings = df_ratings.copy()

In [222]:
cleaned_ratings = cleaned_ratings.merge(number_of_user_ratings, on='user')
cleaned_ratings = cleaned_ratings.merge(number_of_book_ratings, on='isbn')

In [223]:
# clean data from ratings dataset
# remove users with less than 200 ratings
# remove books with less than 100 ratings 
cleaned_ratings = cleaned_ratings.loc[(cleaned_ratings["no_of_book_ratings"]>=100) & (cleaned_ratings["no_of_ratings"]>=200)].sort_values(by=['user'])
cleaned_ratings

,user,isbn,rating,no_of_ratings,no_of_book_ratings
50241,254,0316776963,0.0,314,340
200746,254,0441569595,0.0,314,179
200493,254,0441003257,0.0,314,168
114703,254,0446605239,0.0,314,465
132734,254,0399146431,9.0,314,117
...,...,...,...,...,...
102685,278418,0312974256,0.0,4533,103
167419,278418,0671617028,0.0,4533,146
168518,278418,0671758896,0.0,4533,162
172963,278418,1559029838,0.0,4533,180


In [224]:
# add your code here - consider creating a new cell for each section of code
# combine with book dataframe
combined_cleaned_df = pd.merge(cleaned_ratings, df_books, on='isbn')

In [225]:
combined_cleaned_df.sort_values(by=["user"])

,user,isbn,rating,no_of_ratings,no_of_book_ratings,title,author
0,254,0316776963,0.0,314,340,Me Talk Pretty One Day,David Sedaris
1838,254,080410753X,0.0,314,273,The Kitchen God's Wife,Amy Tan
1938,254,051513287X,0.0,314,237,Face the Fire (Three Sisters Island Trilogy),Nora Roberts
2037,254,0804106304,0.0,314,519,The Joy Luck Club,Amy Tan
2209,254,0316569321,0.0,314,265,White Oleander : A Novel,Janet Fitch
...,...,...,...,...,...,...,...
24283,278418,0671758896,0.0,4533,162,"Loves Music, Loves to Dance",Mary Higgins Clark
33590,278418,0446364193,0.0,4533,298,Along Came a Spider (Alex Cross Novels),James Patterson
42485,278418,0380018179,0.0,4533,216,The Thorn Birds,Colleen McCullough
37906,278418,0451160525,0.0,4533,148,"The Gunslinger (The Dark Tower, Book 1)",Stephen King


In [226]:
# remove duplicates
combined_cleaned_df = combined_cleaned_df.drop_duplicates(['title', 'user'])

In [227]:
data_matrix = combined_cleaned_df.pivot(values='rating', index='title', columns='user').fillna(0)
data_matrix

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [228]:
# make and train model
knn = NearestNeighbors(metric='cosine', algorithm='auto')
knn.fit(data_matrix)

NearestNeighbors(metric='cosine')

In [229]:
# function to return recommended books - this will be tested
def get_recommends(book = "", k=5):
  B = data_matrix[data_matrix.index == book]
  # convert to numpy to pass data
  B.to_numpy().reshape(-1,1)

  distances, indices = knn.kneighbors(B, n_neighbors=k+1)

  recommendations = sorted(zip(indices.squeeze().tolist(), distances.squeeze().tolist()), key=lambda x: x[1])[::-1]

  recommended_books = [
        book,
        []
    ]
  for idx, dist in recommendations[:k]:
        recommended_books[1].append(
            (data_matrix.index[idx], dist)
        )
  return recommended_books

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [230]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [("I'll Be Seeing You", 0.8016210794448853), ('The Weight of Water', 0.7708583474159241), ('The Surgeon', 0.7699410915374756), ('I Know This Much Is True', 0.7677075266838074), ('The Lovely Bones: A Novel', 0.7234864234924316)]]
You passed the challenge! 🎉🎉🎉🎉🎉
